In [17]:
# 自动切换显卡配置

import os
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import torch
import torch_directml
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

print(f"CUDA{'可用' if torch.cuda.is_available() else '不可用'}")
print(f"directml{'可用' if torch_directml.is_available() else '不可用'}")

if (torch.cuda.is_available()):
    device = torch.device("cuda")
elif (torch_directml.is_available()):
    print(torch_directml.device_name(0))
    device = torch_directml.device(0)
else:
    device = torch.device("cpu")

CUDA不可用
directml可用
AMD Radeon RX 6650 XT 


In [18]:
# 数据预处理
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ]),
}

# 数据集路径
data_dir = ""  # 换成你的路径

# 加载数据
image_datasets = {
    x: datasets.ImageFolder(os.path.join(data_dir, x), transform=data_transforms[x])
    for x in ['train', 'val', 'test']
}

dataloaders = {
    x: DataLoader(image_datasets[x], batch_size=32, shuffle=(x == 'train'))
    for x in ['train', 'val', 'test']
}

# 类别数
num_classes = len(image_datasets['train'].classes)
print(f"检测到类别数量：{num_classes}")

# 使用预训练的ResNet50
model = models.resnet50(pretrained=True)

# 替换最后一层
model.fc = nn.Linear(model.fc.in_features, num_classes)

model = model.to(device)

# 损失函数与优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# 训练函数
def train_model(model, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        print("-" * 10)

        for phase in ['train', 'val']:
            model.train() if phase == 'train' else model.eval()
            running_loss, running_corrects = 0.0, 0

            for inputs, labels in tqdm(dataloaders[phase]):
                inputs, labels = inputs.to(device), labels.to(device)

                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(image_datasets[phase])
            epoch_acc = running_corrects.double() / len(image_datasets[phase])
            print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

    return model

# 开始训练
model = train_model(model, criterion, optimizer, num_epochs=10)


检测到类别数量：31
Epoch 1/10
----------


100%|██████████| 275/275 [04:53<00:00,  1.07s/it]


train Loss: 0.8527 Acc: 0.7837


100%|██████████| 86/86 [00:20<00:00,  4.26it/s]


val Loss: 0.2157 Acc: 0.9469
Epoch 2/10
----------


...

val Loss: 0.0417 Acc: 0.9913
Epoch 10/10
----------


100%|██████████| 275/275 [04:42<00:00,  1.03s/it]


train Loss: 0.0490 Acc: 0.9857


100%|██████████| 86/86 [00:19<00:00,  4.36it/s]

val Loss: 0.0555 Acc: 0.9869


In [19]:
# 数据预处理
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ]),
}

# 数据集路径（替换为你自己的路径）
data_dir = ""

# 加载数据
image_datasets = {
    x: datasets.ImageFolder(os.path.join(data_dir, x), transform=data_transforms[x])
    for x in ['train', 'val', 'test']
}

dataloaders = {
    x: DataLoader(image_datasets[x], batch_size=32, shuffle=(x == 'train'))
    for x in ['train', 'val', 'test']
}

# 类别数
num_classes = len(image_datasets['train'].classes)
print(f"检测到类别数量：{num_classes}")

# 使用预训练的 ResNet50
model = models.resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

# 损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# 模型保存目录
save_dir = "./checkpoint"
os.makedirs(save_dir, exist_ok=True)

# 训练函数
def train_model(model, criterion, optimizer, num_epochs=20, save_interval=5):
    for epoch in range(1, num_epochs + 1):
        print(f"\nEpoch {epoch}/{num_epochs}")
        print("-" * 30)

        for phase in ['train', 'val']:
            model.train() if phase == 'train' else model.eval()
            running_loss, running_corrects = 0.0, 0

            for inputs, labels in tqdm(dataloaders[phase], desc=phase):
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(image_datasets[phase])
            epoch_acc = running_corrects.double() / len(image_datasets[phase])
            print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

        # 每 save_interval 轮保存一次
        if epoch % save_interval == 0:
            checkpoint_path = os.path.join(save_dir, f"resnet50_epoch{epoch}.pth")
            torch.save(model.state_dict(), checkpoint_path)
            print(f"✅ 模型已保存到 {checkpoint_path}")

    # 最终保存一次
    final_path = os.path.join(save_dir, "resnet50_final.pth")
    torch.save(model.state_dict(), final_path)
    print(f"\n🎉 最终模型已保存到 {final_path}")

    return model

# 开始训练
model = train_model(model, criterion, optimizer, num_epochs=30, save_interval=5)

# 测试函数
def evaluate(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in tqdm(dataloader, desc="Testing"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            correct += torch.sum(preds == labels)
            total += labels.size(0)
    print(f"\n📊 Test Accuracy: {correct.double() / total:.4f}")

# 评估模型
evaluate(model, dataloaders['test'])


检测到类别数量：31

Epoch 1/30
------------------------------


train: 100%|██████████| 275/275 [04:50<00:00,  1.06s/it]


train Loss: 0.8945 Acc: 0.7706


val: 100%|██████████| 86/86 [00:20<00:00,  4.24it/s]


val Loss: 0.2436 Acc: 0.9360

Epoch 2/30
------------------------------


...

train Loss: 0.0482 Acc: 0.9868


val: 100%|██████████| 86/86 [00:20<00:00,  4.18it/s]


val Loss: 0.0643 Acc: 0.9844
✅ 模型已保存到 ./checkpoint\resnet50_epoch30.pth

🎉 最终模型已保存到 ./checkpoint\resnet50_final.pth


Testing: 100%|██████████| 55/55 [00:15<00:00,  3.66it/s]


📊 Test Accuracy: 0.9761


In [3]:
import os
import torch
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from tqdm import tqdm
from datetime import datetime

# ========== 基础路径 ==========
base_dir = ""
checkpoint_dir = os.path.join(base_dir, "checkpoint")
test_dir = os.path.join(base_dir, "test")
log_path = os.path.join(base_dir, "evaluation_log.txt")

# ========== 数据预处理 ==========
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# ========== 加载测试数据集 ==========
test_dataset = datasets.ImageFolder(test_dir, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
num_classes = len(test_dataset.classes)

# ========== 函数：评估模型 ==========
def evaluate_model(model_path):
    model = models.resnet50(pretrained=False)
    model.fc = torch.nn.Linear(model.fc.in_features, num_classes)
    
    # 加载模型参数
    state_dict = torch.load(model_path, map_location=device, weights_only=False)
    model.load_state_dict(state_dict)

    model = model.to(device)
    model.eval()

    correct = 0
    total = 0

    # ✅ 使用 tqdm 包裹测试数据
    test_loop = tqdm(test_loader, desc=f"测试中: {os.path.basename(model_path)}", leave=False)
    
    with torch.no_grad():
        for inputs, labels in test_loop:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            correct += torch.sum(preds == labels)
            total += labels.size(0)
    
    accuracy = correct.double() / total
    return accuracy.item()



# ========== 主逻辑 ==========
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
results = []

# 遍历所有 .pth 文件
for filename in sorted(os.listdir(checkpoint_dir)):
    if filename.endswith(".pth"):
        model_path = os.path.join(checkpoint_dir, filename)
        print(f"🔍 正在测试模型：{filename}")
        acc = evaluate_model(model_path)
        result_str = f"{filename} | 准确率: {acc:.4f} ({acc*100:.2f}%)"
        print("✅ " + result_str)
        results.append(result_str)

# ========== 写入日志文件 ==========
with open(log_path, "w", encoding="utf-8") as f:
    f.write(f"模型评估日志 - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write("=" * 50 + "\n")
    for line in results:
        f.write(line + "\n")

print(f"\n📁 所有结果已写入日志文件: {log_path}")


🔍 正在测试模型：resnet50_epoch10.pth


✅ resnet50_epoch10.pth | 准确率: 0.9727 (97.27%)
🔍 正在测试模型：resnet50_epoch15.pth


✅ resnet50_epoch15.pth | 准确率: 0.9795 (97.95%)
🔍 正在测试模型：resnet50_epoch20.pth


✅ resnet50_epoch20.pth | 准确率: 0.9864 (98.64%)
🔍 正在测试模型：resnet50_epoch25.pth


✅ resnet50_epoch25.pth | 准确率: 0.9767 (97.67%)
🔍 正在测试模型：resnet50_epoch30.pth


✅ resnet50_epoch30.pth | 准确率: 0.9761 (97.61%)
🔍 正在测试模型：resnet50_epoch5.pth


✅ resnet50_epoch5.pth | 准确率: 0.9818 (98.18%)
🔍 正在测试模型：resnet50_final.pth


✅ resnet50_final.pth | 准确率: 0.9761 (97.61%)

📁 所有结果已写入日志文件: evaluation_log.txt
